# Data Class

In [8]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #Score of 4 or 5
            return Sentiment.POSITIVE
        
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        

# Load Data

In [5]:
import json

file_name = './books_small_10000.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

# Prep Data

In [23]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [26]:
# Evenly distribute the training data
train_container.evenly_distribute()
test_container.evenly_distribute()

X_train = train_container.get_text()
y_train = train_container.get_sentiment()

X_test = test_container.get_text()
y_test = test_container.get_sentiment()

print(y_train.count(Sentiment.NEGATIVE))
print(y_train.count(Sentiment.POSITIVE))
print(y_test.count(Sentiment.NEGATIVE))
print(y_test.count(Sentiment.POSITIVE))

436
436
208
208


# Bags of Words Vectorization

In [35]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


# The problem with the CountVectorizer is that it weights all words equally.
# vectorizer = CountVectorizer()

# Instead, let's use the TfidfVectorizer which weights words based on their importance
vectorizer = TfidfVectorizer()

# fit and transform training data
X_train_vectors = vectorizer.fit_transform(X_train)

# transform test data
X_test_vectors = vectorizer.transform(X_test)



# Classification

### Linear SVM (Support Vector Machine)

In [43]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(X_train_vectors, y_train)

print(X_test[0])
clf_svm.predict(X_test_vectors[0])

I expected personal stories of women marrying soldiers.  This book takes forever to set up the characters and then very slowly tries to tell a story......not too believable.


array(['NEGATIVE'], dtype='<U8')

### Decision Tree

In [44]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()

clf_dec.fit(X_train_vectors, y_train)

clf_dec.predict(X_test_vectors[0])

array(['NEGATIVE'], dtype='<U8')

### Naive Bayes

In [45]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()

clf_gnb.fit(X_train_vectors.toarray(), y_train)

clf_gnb.predict(X_test_vectors[0].toarray())

array(['NEGATIVE'], dtype='<U8')

### Logistic Regression

In [46]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()

clf_log.fit(X_train_vectors, y_train)

clf_log.predict(X_test_vectors[0])

array(['NEGATIVE'], dtype='<U8')

## Evaluation

In [47]:
# MEAN ACCURACY
# Comparison of test vectors and test data
# .score() method returns the mean accuracy on the given test data and labels.
print(clf_svm.score(X_test_vectors, y_test))
print(clf_dec.score(X_test_vectors, y_test))
print(clf_gnb.score(X_test_vectors.toarray(), y_test))
print(clf_log.score(X_test_vectors, y_test))

0.8076923076923077
0.6442307692307693
0.6610576923076923
0.8028846153846154


In [48]:
# F1 SCORE
# The F1 score can be interpreted as a weighted average of the precision and recall,
# where an F1 score reaches its best value at 1 and worst score at 0.
from sklearn.metrics import f1_score

f1_score(y_test, clf_svm.predict(X_test_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])

array([0.80582524, 0.80952381])

In [49]:
test_set = ['great for the price', 'horrible, never buy', 'not bad']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

## Tuning our model (with Grid Search)

In [51]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(X_train_vectors, y_train)


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [52]:
print(clf.score(X_test_vectors, y_test))

0.8197115384615384


## Saving Model